One png at a time, configure fields for each one of them and save output

In [1]:
import json
from fuzzywuzzy import fuzz

/home/ssddevavrat/anaconda3/envs/ML_env/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def aligned_horizontal_top(box1, box2, threshold=0.01):
    if abs(box1['Top'] - box2['Top']) < threshold:
        return True
    return False

def aligned_horizontal_bottom(box1, box2, threshold=0.01):
    if abs((box1['Top']-box1['Height']) - (box2['Top']-box2['Height'])) < threshold:
        return True
    return False

def aligned_vertical_left(box1, box2, threshold=0.01):
    if abs(box1['Left'] - box2['Left']) < threshold:
        return True
    return False

def contains_horizontal(box1, box2):
    mid_x = box2['Left'] + (box2['Width'] / 2)
    mid_y = box2['Top'] - (box2['Height'] / 2)
    if  box1['Top'] > mid_y and mid_y > box1['Top']-box1['Height']:
        return True
    return False

def contains_vertical(box1, box2):
    mid_x = box2['Left'] + (box2['Width'] / 2)
    mid_y = box2['Top'] - (box2['Height'] / 2)
    if box1['Left'] < mid_x and mid_x < box1['Left']+box1['Width']:
        return True
    return False

def aligned_vertical_right(box1, box2, threshold=0.01):
    if abs((box1['Left']+box1['Width']) - (box2['Left']+box2['Width'])) < threshold:
        return True
    return False

def letter_fraction(word):
    letter = 0.0001
    total = 0.0001
    for char in word:
        if char.isalpha():
            letter += 1
            total += 1
        if char.isdigit():
            total += 1
    return letter / total

def number_fraction(word):
    number = 0.0001
    total = 0.0001
    for char in word:
        if char.isdigit():
            number += 1
            total += 1
        if char.isalpha():
            total += 1
    return number / total

In [4]:
def read_json(name):
    with open(name, 'r') as file:
        api_output = file.read()
        data = json.loads(api_output)
    return data

def remove_PHP(data):
    table = []
    count = 0
    # text = data['raw_response']['Blocks']
    text = data['Blocks']
    for i in range(1, len(text)):
        box = text[i]['Geometry']['BoundingBox']
        polygon = text[i]['Geometry']['Polygon']
        word = text[i]['Text']
        if word == 'PHP':
            continue
        table.append([count,word,box,polygon,abs(polygon[0]['Y']-polygon[1]['Y']),abs(polygon[0]['X']-polygon[3]['X'])])
    return table

def remove_Hrs(data):
    table = []
    save_hrs = []
    for i in range(0, len(data)):
        if data[i][1] == 'Hrs.':
            save_hrs.append(data[i])
            # print (i, data[i])
    
    for i in range(0, len(data)):
        flag = 0
        for j in range (0, len(save_hrs)):
            if aligned_vertical_left(data[i][2], save_hrs[j][2]) or aligned_vertical_right(data[i][2], save_hrs[j][2]):
                flag = 1
                continue
        if flag == 0:
            table.append(data[i])
    
    return table

def find_headings(data):
    list = [] 
    for textbox in data:
        if textbox[1].lower() == 'earnings':
            list.append((textbox[1], textbox[2]))
            break
    
    for textbox in data:
        if textbox[1].lower() == 'deductions':
            list.append((textbox[1], textbox[2]))
            break
    
    for textbox in data:
        if textbox[1].lower() == 'year-to-date summary':
            list.append((textbox[1], textbox[2]))
            break
    
    return list

def horizontal_construct(table):
    key_value = []
    for i in range(0, len(table)):
        temp = []
        for j in range(i+1, len(table)):
            # if ((aligned_horizontal_bottom(table[i][2], table[j][2]) or aligned_horizontal_top(table[i][2], table[j][2]))):
            if contains_horizontal(table[i][2], table[j][2]) or contains_horizontal(table[j][2], table[i][2]):
                temp.append(table[j])
        key_value.append([table[i], temp])
    return key_value

def sort_horizontal(key_values: list):
    total = 0
    printed = 0
    sorted_key_values = []
    for item in key_values:
        sorted_items = sorted(item[1], key=lambda x: x[2]['Left'])
        temp = []  
        for stuff in sorted_items:
            total += 1
            left_difference = item[0][2]['Left'] - stuff[2]['Left']
            right_difference = (item[0][2]['Left']+item[0][2]['Width']) - (stuff[2]['Left']+stuff[2]['Width'])
            if left_difference < 0 and right_difference < -0.02:
                printed += 1
                temp.append(stuff)
        sorted_key_values.append([item[0], temp])
    # print("Total pairs before elimination: ", total)
    # print("Total pairs after: ", printed)
    return sorted_key_values

def find_value(sorted_key_values):
    final = []
    for item in sorted_key_values:
        ans = "NO_VALUE_FOUND"
        for _ in item[1]:
            if number_fraction(_[1]) > 0.5:
                ans = _[1]
            if letter_fraction(_[1]) > 0.5:
                if ans == "NO_VALUE_FOUND":
                    pass
                    # ans = "BROKE_LOOP_AT_" + _[1]
                break
        final.append([item, ans])
    return final

def find_fields(answer, match_list, similarity_threshold=90):
	final_answer = []
	for word in match_list:
		for _ in answer:
			# print(_[0][0][1])
			detected = _[0][0][1].lower()
			if fuzz.ratio(detected, word.lower()) >= similarity_threshold:
			# if detected == word:
				if _[1] != "NO_VALUE_FOUND":
					final_answer.append((_[0][0], _[0][1]))
	return final_answer

Cognizant - A lot of terms aren't detected because of the word "(NT)" or "(T)" along with the label, need to fix this

Logic isn't taking the rightmost number i think, need to fix this

Transcom - I think the first image might be a different format?

In [59]:
# match_list = ("employee number", "period", "basic pay", "undertime", "tardiness", "absences", "vto deduction", "deminimis", "transportation allowance", "rice allowance", "communication allowance", "cvd wfh allow", "hdmf premium", "phic premium", "sss premium", "sss provident", "pagcal", "pagln", "sscal", "sssal", "advance loan", "taxable transpo allowance", "net pay", "withholding tax", "employee number", "period", "period", "basic pay", "undertime", "tardiness", "absences", "vto deduction", "deminimis", "transportation allowance", "rice allowance", "communication allowance", "cvd wfh allow", "hdmf premium", "phic premium", "sss premium", "sss provident", "pagcal", "pagln", "sscal", "sssal", "advance loan", "taxable transpo allowance", "net pay", "withholding tax", "employee code", "payment date", "basic salary", "undertime", "tardiness", "absences", "voluntary time off ut", "transportation allowance", "rice allowance", "deminimis", "hdmf contribution", "philhealth contribution", "sss contribution", "sss provident fund", "net pay", "withholding tax")

# congnizant format 1
# match_list = ("employee code", "employee name", "pay period", "basic pay", "basic pay adj", "tardiness", "abscences", "overtime", "clothing allow (NT)", "laundry allow (NT)", "rice allow (NT)", "medical allow (NT)", "other allow (NT)", "Language Allow", "HDMF - Contribution", "Philhealth - Contribution", "SSS - Contribution", "SSS MPF - Contribution", "HDMF calamity loan", "HDMF salary loan", "SSS SLERP", "SSS salary loan", "Global Credit Pro Loan", "NET PAY", "Withholding Tax")

# congnizant format 2
# match_list = ("Employee Code", "Employee Name", "Pay Period", "Basic Pay", "Basic Pay Adj", "Tardiness", "Absences", "WFH Internet Allowance", "Clothing Allow", "Laundry Allow", "Rice Allow", "Medical Allow", "Other Allow", "Language Allow", "HDMF - Contribution", "Philhealth - Contribution", "SSS - Contribution", "SSS MPF - Contribution", "HDMF calamity loan", "HDMF salary loan", "SSS SLERP", "SSS salary loan", "Global Credit Pro Loan", "NET PAY", "Withholding Tax")

#Ibex format 1
# match_list = ("Employee iD", "Employee Name", "Pay Period", "Basic Salary", "Under Time Deduction ", "Lates Deduction", "Absences Deduction", "Laundry Allowance", "Rice Allowance", "Meal Allowance", "Medicine Allowance", "Program Allowance", "WOS Agent Alllowance", "Pag Ibig Contribution", "PhilHealth Contribution", "SSS Contribution", "SSS Employee Mandatory Provident Fund", "Pag Ibig Calamity Loan", "Pag Ibig Loan", "SSS Loan-Calamity", "SSS Loan", "Cash Advance", "NET PAYMENT", "BIR Withholding Tax")

#IntouchCX format 1
# match_list = ("Employee Number", "Period", "Basic Pay", "Undertime", "Tardiness", "Absences", "CLOTHING ALLOWANCE", "LAUNDRY ALLOWANCE", "RICE SUBSIDY", "MEAL ALLOWANCE", "SKILL ALLOWANCE", "DE MINIMIS ALLOWANCE", "PROGRAM ALLOWANCE", "PHIC Premium", "HDMF Premium", "SSS Premium", "SSS Provident", "PAGIBIG CALAMITY", "PAGIBIG LOAN", "SSS CALAMITY", "SSS LOAN", "HMO DEDUCTION", "ADVANCE PAYMENT", "NET PAY", "Withholding Tax")

#IntouchCX format 2
# match_list = ("Employee Number", "Period", "Basic Pay", "Undertime", "Tardiness", "Absences", "CLOTHING ALLOWANCE", "LAUNDRY ALLOWANCE", "RICE SUBSIDY", "MEAL ALLOWANCE", "SKILL ALLOWANCE", "DE MINIMIS ALLOWANCE", "PROGRAM ALLOWANCE", "PHIC Premium", "HDMF Premium", "SSS Premium", "SSS Provident", "PAGIBIG CALAMITY", "PAGIBIG LOAN", "SSS CALAMITY", "SSS LOAN", "HMO DEDUCTION", "ADVANCE PAYMENT", "NET PAY", "Withholding Tax")

#Sagility format 1 
# match_list = ("Employee Number",  "Period",  "Basic Pay",  "Undertime",  "Absences",  "Tardiness",  "Account Premium",  "Rice",  "HDMF Premium",  "PHIC Premium",  "SSS Premium",  "SSS Provident",  "PAGIBIG CALAMITY",  "PAGIBIG LOAN",  "SSS LOAN CALAMITY",  "SSS SALARY LOAN",  "HMO_Y23",  "COMPANY LOAN",  "NET PAY",  "Withholding Tax")

#Sagility format 2
# match_list = ("Employee Number", "Period", "Basic Pay", "Undertime", "Absences", "Tardiness", "Account Premium", "Rice", "HDMF Premium", "PHIC Premium", "SSS Premium", "SSS Provident", "PAGIBIG CALAMITY", "PAGIBIG LOAN", "SSS LOAN CALAMITY (SLERP)", "SSS SALARY LOAN", "HMO_Y23", "COMPANY LOAN", "NET PAY", "Withholding Tax")

#Transcom format 1
# match_list = ("Employee No.", "Employee Name", "Period Covered", "Basic Pay", "Undetim", "Late", "Absent", "LWOP", "VTO", "Trans Allowance", "Rice Subsidy", "Clothing Allowance", "Pag-Ibig", "Philhealth", "SSS Premium", "MPF EE Contribution", "Provident Fund Contribut", "Pag Calamity Ln-Ded", "Pag-Ibig STL-Ded", "SSS Calamity - Ded", "SSS Salary Loan-Ded", "Transcom Care CalamityDed", "Maxicare depend.Deduct CY", "NET PAY", "Withholding Tax")

#Ubiquity format 1 
# match_list = ("Employee iD", "Employee Name", "Pay Period", "Basic pay", "Undertime", "Tardiness", "Absences", "Non Taxable Allowances", "Employee HDMF", "Employee Phil Health", "Employee SSS", "Multi-Purpose Loan", "SSS Calamity Loan", "SSS Salary Loan", "HMO", "Net Earnings", "Withholding Tax")

#WNS Format 1 
match_list = ("Employee ID", "Employee Name", "Payslip for the period", "Basic", "Undertime", "Tardiness", "Unpaid Leave", "Rice Allowance", "Clothing Allowance", "Medical Allowance", "HDMF Employee", "Phil Health Employee", "SSS Employee", "SSS WISP Employee", "HDMF LOAN", "SSS Calamity", "SSS Loan", "Home Credit PHL", "HMO_IC PHL", "NET SALARY (A-B)", "Income tax")

In [66]:
# ENTER NAME OF THE JSON FILE HERE
name = 'WNS_Format1_7'
data = read_json('TEXTRACT_JSONS/'+name+'.json')

for i in range (len(data)):
    # input = data[i]
    input = data
    
	# THIS IS THE LOCATION OF THE OUTPUT FILE
    output_file = f'TEXTRACT_OUTPUTS/{name}.txt'

    # table = remove_PHP(data[i])
    table = remove_PHP(data)
    table_without_Hrs = remove_Hrs(table)

    titles = find_headings(table_without_Hrs)
    
    key_values = horizontal_construct(table_without_Hrs)
    sorted_key_values = sort_horizontal(key_values)
    answer = find_value(sorted_key_values)
    final_answer = find_fields(answer, match_list)

    # print(titles)
    
    with open(output_file, 'w') as file:
        for _ in final_answer:
            if letter_fraction(_[0][1])>0.5:
                height = _[0][2]['Top']
                box_height = 0
                ans = []
                for box in titles:
                    box_bound = box[1]
                    if box_bound['Top'] > height: # title is below the identified word
                        continue
                    if contains_vertical(box_bound, _[0][2]) or contains_vertical(_[0][2], box_bound):
                        if box_bound['Top'] > box_height: # title is lowest possible
                            box_height = box_bound['Top']
                            ans = (box[0], _[0][1], _[1][0][1])
                if ans:
                    print(ans, file=file)
                else:
                    print (_[0][1], _[1][0][1], file=file)

Typing module

In [6]:
print(data["Blocks"])

[{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 1.0, 'Height': 1.0, 'Left': 0.0, 'Top': 0.0}, 'Polygon': [{'X': 9.92603645499912e-07, 'Y': 0.0}, {'X': 1.0, 'Y': 7.409223599097459e-07}, {'X': 1.0, 'Y': 1.0}, {'X': 0.0, 'Y': 1.0}]}, 'Id': '960e1b34-d89d-48de-aaeb-4199f66847e7', 'Relationships': [{'Type': 'CHILD', 'Ids': ['54b60a2c-0aa8-4534-94ff-9710ffe446e9', 'c0ac86a2-d539-431c-8436-71e0f13ee358', 'cb9b378d-8973-490d-9843-eef90efe4437', '71c92731-3e2f-4bf0-9c7f-f75281c3641b', 'b3dde98e-efd3-411b-b707-dff119d59ef8', '2f5f5bbc-f5f3-4738-b911-4d7fb8e01d0c', '0dbcaea4-8c69-4ee7-abc5-1b9b32053e87', '85dc164a-836a-447f-a243-23e889ba2cc2', '4aa1c994-5ab3-4d89-8984-3dacb31f11db', '4e089bc6-8be5-47a8-b953-483453747c85', '491f325c-7760-4151-9e9d-122ffc814991', 'f8026b99-f8b9-4821-ae64-51231a43ad52', 'bef7d2c9-d136-4377-969d-bb306249e298', '299709de-d53a-4ca6-83cc-81ffab423336', '476f367c-bb05-4000-90fc-7ad11a5dfb57', 'a42986ce-cf7e-4f91-a6f6-23f83b02e922', 'fb58bb36-c923-4e65-b1bb-f